In [60]:
import sys,csv, json, re
#from settings import *
def load_settings(fileN):
    if fileN:
        with open(fileN, 'r') as f:
            return json.loads(f.read())
    else:
        return 0
def print_settings(settings):
    for key in settings:
        print(key, ": ", settings[key])
    
settings = load_settings('./settings.json')
print_settings(settings)

no_files :  2
files :  ['./Inventory.csv', './ItemMasterStandard.xlsx-Sheet1.csv']
file_1_cols :  [1, 2, 3]
file_2_cols :  [1, 4, 5]


In [22]:
import pandas as pd
# print out columns in both df
for x in range(settings['no_files']):
    fname = settings['files'][x]
    print(fname)
    df = pd.read_csv(fname)
    #print(df.to_string())
    print(list(df.columns))

./Inventory.csv
['id', 'detailed_type', 'categ_id', 'default_code', 'name', 'sale_ok', 'purchase_ok', 'uom_id', 'uom_po_id', 'route_ids']
./ItemMasterStandard.xlsx-Sheet1.csv
['Part Number', 'Description', 'Category', 'Item Type', 'Unit', 'Replacement Cost', 'Vendor Lead Time', 'QC/Transit Lead Time', 'Buffer Days', 'Quality Control Group', 'Material Cost', 'Buyer']


In [23]:
# separate columns from df
columns = ['Part Number', 'Description', 'Category','Quality Control Group', 'Material Cost']
df_itemmaster = df[columns]
df_itemmaster

,Part Number,Description,Category,Quality Control Group,Material Cost
0,1 Sales,1 Sales,Other,NaN,0.00
1,1 Sales of product Income,1 Sales of product Income,Other,NaN,0.00
2,3% Credit Card Fee,3% Credit Card Fee,Other,NaN,0.00
3,3% Credit Card Processing Fee,3% Credit Card Processing Fee,Other,NaN,0.00
4,ACC001,Sales (General),Other,NaN,0.00
...,...,...,...,...,...
12686,zDELETED_for_tick_2022-24757,Item added for ticket 2022-24757. Used to assi...,Other,Multiple,0.00
12687,zDELETEME,zDELETEME,Other,Multiple,0.00
12688,NaN,NaN,NaN,NaN,NaN
12689,NaN,NaN,NaN,NaN,NaN


In [68]:
# analyze columns
for col in columns:
    print(col, df_itemmaster[col].nunique())
    #for uniq in df_itemmaster[col].unique():
    #    print(uniq)
    # list of uniques per column
    cuniq = list(df_itemmaster[col].unique())
    #for item in cuniq:
    #    print(item, list(df_itemmaster[col]).count(item))

Part Number 12689
Description 12493
Category 40
Quality Control Group 1296
Material Cost 2470


In [69]:
# show unique values of column
#list(df_itemmaster['Quality Control Group'].unique())

In [47]:
df_itemmaster['Quality Control Group']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
12686    Multiple
12687    Multiple
12688         NaN
12689         NaN
12690         NaN
Name: Quality Control Group, Length: 12691, dtype: object

In [36]:
# lets start with some rules with those columns so we can have an idea of what should be the info
# 'Part Number', 'Description', 'Category','Quality Control Group', 'Material Cost', no NAs
df_itemmaster_nonas = df_itemmaster.dropna()
df_itemmaster_nonas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5559 entries, 68 to 12687
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Part Number            5559 non-null   object
 1   Description            5559 non-null   object
 2   Category               5559 non-null   object
 3   Quality Control Group  5559 non-null   object
 4   Material Cost          5559 non-null   object
dtypes: object(5)
memory usage: 260.6+ KB


In [37]:
df_itemmaster_nonas.head()

,Part Number,Description,Category,Quality Control Group,Material Cost
68,CBA002,Bottles 1 Gallon Black HDPE Smooth Side Jar (1...,Bottle (B),07 Packaging Component,1.23
69,CBA003,Bottles 1 Gallon Black HDPE (110-400) Indented...,Bottle (B),07 Packaging Component,0.91
70,CBA004,Bottles 1 Gallon Black Smooth Packer (110-400),Bottle (B),07 Packaging Component,1.12
71,CBA005,Bottles 1 Gallon Black HDPE (110-400) Canyon P...,Bottle (B),07 Packaging Component,0.88
72,CBA006,Bottles 1 Gallon Clear PET,Bottle (B),07 Packaging Component,0.00


In [42]:
# 'Part Number', 'Description', 'Category','Quality Control Group', 'Material Cost', no NAs
df_itemmaster_nonas_cols = df_itemmaster.dropna(subset=['Part Number', 'Description', 'Category','Quality Control Group'])
df_itemmaster_nonas_cols.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5559 entries, 68 to 12687
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Part Number            5559 non-null   object
 1   Description            5559 non-null   object
 2   Category               5559 non-null   object
 3   Quality Control Group  5559 non-null   object
 4   Material Cost          5559 non-null   object
dtypes: object(5)
memory usage: 260.6+ KB


In [43]:
df_itemmaster_nonas_cols.head()

,Part Number,Description,Category,Quality Control Group,Material Cost
68,CBA002,Bottles 1 Gallon Black HDPE Smooth Side Jar (1...,Bottle (B),07 Packaging Component,1.23
69,CBA003,Bottles 1 Gallon Black HDPE (110-400) Indented...,Bottle (B),07 Packaging Component,0.91
70,CBA004,Bottles 1 Gallon Black Smooth Packer (110-400),Bottle (B),07 Packaging Component,1.12
71,CBA005,Bottles 1 Gallon Black HDPE (110-400) Canyon P...,Bottle (B),07 Packaging Component,0.88
72,CBA006,Bottles 1 Gallon Clear PET,Bottle (B),07 Packaging Component,0.00


In [48]:
# 'Part Number', 'Description', 'Category','Quality Control Group', 'Material Cost', no NAs
df_itemmaster_nonas_cols_code = df_itemmaster.dropna(subset=['Part Number', 'Description', 'Category'])
df_itemmaster_nonas_cols_code.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12688 entries, 0 to 12687
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Part Number            12688 non-null  object
 1   Description            12688 non-null  object
 2   Category               12688 non-null  object
 3   Quality Control Group  5559 non-null   object
 4   Material Cost          12688 non-null  object
dtypes: object(5)
memory usage: 594.8+ KB


In [64]:
import re
pattern = r'^[A-Z]{1,4}[0-9]{1,4}'
#df_filtered = df_itemmaster_nonas_cols_code[df_itemmaster_nonas_cols_code['Part Number'].str.len()<9]
df_filtered = df_itemmaster_nonas_cols_code[df_itemmaster_nonas_cols_code['Part Number'].str.contains(pattern)]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12438 entries, 4 to 12626
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Part Number            12438 non-null  object
 1   Description            12438 non-null  object
 2   Category               12438 non-null  object
 3   Quality Control Group  5543 non-null   object
 4   Material Cost          12438 non-null  object
dtypes: object(5)
memory usage: 583.0+ KB


In [65]:
df_filtered

,Part Number,Description,Category,Quality Control Group,Material Cost
4,ACC001,Sales (General),Other,NaN,0.00
5,ACC002,Inventory Shrinkage,Other,NaN,0.00
6,ACC003,Shipping/Delivery Income,Other,NaN,0.00
7,ACC004,Shipping/Fulfillment Expense,Other,NaN,0.00
8,ACC005,Advertising Expense,Other,NaN,0.00
...,...,...,...,...,...
12622,TEL008,"Minerals (Sb, Ca, Cr, Co, Cu, Fe, Li, Mg, Mn, ...",Fees/Outside Services (F),NaN,0.00
12623,TEL009,Miscellaneous,Fees/Outside Services (F),NaN,0.00
12624,TEL010,Protein,Fees/Outside Services (F),NaN,0.00
12625,TEL012,Organic Certification,Fees/Outside Services (F),NaN,0.00


In [67]:
# set to csv
df_filtered.to_csv('filtered_nonas_cols_code.csv', index=False)